In [1]:
import pandas as pd
import numpy as np

In [15]:
# 1 m2 to ft2
m2_to_ft2 = 10.764

# url for different datasets
sales_consideration_url = "http://www.rvd.gov.hk/datagovhk/7.3.csv"
secondary_avg_price_district_url = "http://www.rvd.gov.hk/datagovhk/1.2M.csv"


### Primary and secondary sales amount and consideration by year

In [8]:
sales_considerations_df = pd.read_csv(sales_consideration_url, skiprows = 1, encoding = "utf-8")

In [ ]:
# done
sales_considerations_df.head()

,Month,Primary Sales Number,Primary Sales Consideration,Secondary Sales Number,Secondary Sales Consideration
0,01-2002,2718,8218,5129,9966
1,02-2002,1506,4305,4189,8070
2,03-2002,1709,4190,3741,6795
3,04-2002,2543,6516,4866,8739
4,05-2002,2257,5844,5068,11097


In [77]:
sales_considerations_df.to_csv("sales_numbers_with_considerations.csv", index = False, encoding = "utf-8")

### Secondary consideration by district

In [16]:
secondary_avg_price_district_df = pd.read_csv(secondary_avg_price_district_url, skiprows = 1, encoding = "utf-8")

In [17]:
secondary_avg_price_district_df.head()

,Month,Class A Hong Kong,Class A Hong Kong - Remarks,Class A Kowloon,Class A Kowloon - Remarks,Class A New Territories,Class A New Territories - Remarks,Class B Hong Kong,Class B Hong Kong - Remarks,Class B Kowloon,...,Class D Kowloon,Class D Kowloon - Remarks,Class D New Territories,Class D New Territories - Remarks,Class E Hong Kong,Class E Hong Kong - Remarks,Class E Kowloon,Class E Kowloon - Remarks,Class E New Territories,Class E New Territories - Remarks
0,01-1999,42663,NaN,36842,NaN,36679,NaN,50141,NaN,37945,...,56292,NaN,50457,NaN,72978,NaN,62551,Z,50968,NaN
1,02-1999,43068,NaN,35834,NaN,36802,NaN,47950,NaN,38428,...,54817,NaN,46759,NaN,66326,NaN,59774,Z,45945,NaN
2,03-1999,42683,NaN,36731,NaN,36992,NaN,51060,NaN,38254,...,51646,NaN,50737,NaN,74608,NaN,61790,Z,46120,NaN
3,04-1999,43223,NaN,36543,NaN,36485,NaN,50189,NaN,39445,...,48628,NaN,43915,NaN,73489,NaN,75745,Z,46953,Z
4,05-1999,43316,NaN,36449,NaN,38035,NaN,50650,NaN,37945,...,50312,NaN,48495,NaN,78077,NaN,86018,Z,50454,NaN


In [ ]:
# include only columns with consideration without remarks columns
col_list = []

for col in secondary_avg_price_district_df.columns:
    if 'remarks' not in col.lower():
        col_list.append(col)

['Month', 'Class A Hong Kong', 'Class A Kowloon', 'Class A New Territories', 'Class B Hong Kong', 'Class B Kowloon', 'Class B New Territories', 'Class C Hong Kong', 'Class C Kowloon', 'Class C New Territories', 'Class D Hong Kong', 'Class D Kowloon', 'Class D New Territories', 'Class E Hong Kong', 'Class E Kowloon', 'Class E New Territories']


In [24]:
result_df = secondary_avg_price_district_df[col_list]

In [67]:
# unpivot the dataframe to have a long format
result_melt_df = pd.melt(result_df, id_vars = ['Month'], var_name = 'property_category', value_name = 'secondary_consideration')

In [68]:
# separate property category into class and district columns, and remove prefix and suffix spaces
result_melt_df['district'] = result_melt_df['property_category'].str[7:].str.strip()
result_melt_df['property_category'] = result_melt_df['property_category'].str[:7].str.strip()

In [69]:
# convert the secondary consideration to numeric and replace '-' with 0
result_melt_df['secondary_consideration'] = result_melt_df['secondary_consideration'].replace('-', 0)

# convert the consideration from per m2 to per ft2
result_melt_df['secondary_consideration'] = result_melt_df['secondary_consideration'].astype('float') / m2_to_ft2

In [70]:
# rearrange the columns
secondary_avg_price_district_output_df = result_melt_df[['Month','district','property_category','secondary_consideration']]

In [76]:
secondary_avg_price_district_output_df.to_csv('secondary_avg_price_district.csv', index = False, encoding = 'utf-8-sig')